## Gaussian Discriminant Analysis

with same covariance $\Sigma_1 = \Sigma_2 = \Sigma_3 ... = \Sigma_3$

In [1]:
import pandas as pd
import numpy as np

# To reset the printoptions
# np.set_printoptions(edgeitems=3,infstr='inf',linewidth=75, nanstr='nan', precision=8,suppress=False, threshold=1000, formatter=None)

np.set_printoptions(precision=4, suppress=True)

#np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

### For multi-class label y ={0, 1, 2, ...}

In [2]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
y = dataset.target
#y=(y>0).astype(int) 

target_names = list(dataset.target_names)
print(target_names)

['setosa', 'versicolor', 'virginica']


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112, 4), (38, 4), (112,), (38,))

In [5]:
class GaussianDiscAnalysis:
    """
    Fits a Linear Discriminant Analysis Model for multi-class data
    
    lamd : Preconditioner; for matrix inversion; default:1e-10
    """
    def __init__(self, lambd=1e-10):
        self.lambd = lambd
        
    def compute_phi(self, y):
        n = len(y)
        phi = dict()
        for idx in range(self.num_classes):
            phi[idx] = (1/n) * np.sum(y==self.classes[idx])
        return phi
    
    def compute_mu(self, X, y):
        mu_dict = dict()
        for idx in range(self.num_classes):
            # Add mu for each class
            mu_dict[idx] = np.sum(X[y==self.classes[idx]], axis=0)/ np.sum(y==self.classes[idx])
        return mu_dict

    def compute_sigma(self, X, y):
        n = len(X)
        #y = y.reshape(-1,1)
        Xmu = X.copy()
        for idx in range(self.num_classes):
            Xmu = Xmu \
              - self.mu[idx]*np.ones_like(Xmu)*(y==self.classes[idx]).reshape(-1,1)
        return (1/n) * Xmu.T@Xmu
    
    
    def compute_Pxyi(self, X, idx):
        """Probability of X given y"""
        d = X.shape[1]
        sigma_inv = np.linalg.inv(self.sigma)
        det_sigma = np.linalg.det(self.sigma)
        #mu_i = mu(X, y, idx)
        Pxi = (1/((2*np.pi)**(d/2))) \
                *(1/(det_sigma**0.5)) \
                * np.exp(- 0.5*np.sum(((X-self.mu[idx])@sigma_inv)*(X-self.mu[idx]), axis=1))
    #     Pxi = np.log(1) \
    #             - np.log((2*np.pi)**(m/2)) \
    #             - np.log(np.sqrt(det_sigma)) \
    #             - np.sum(((X-mu_i)@sigma_inv)*(X-mu_i), axis=1)
        return Pxi
    
    def fit(self, X, y):
        """Computes mean, covariance and proabilities of y (phi)"""
        d = X.shape[1]
        self.classes = np.unique(y)
        self.num_classes = len(self.classes)
        self.mu = self.compute_mu(X, y)
        self.sigma = self.compute_sigma(X, y) + self.lambd*np.eye(d,d)
        self.phi = self.compute_phi(y)
        
    def predict_proba(self, X):
        """Computes the probability of example belonging to that class"""
        n = len(X)
        Pyi = np.zeros((n, self.num_classes))
        
        for idx in range(self.num_classes):
            #print(self.compute_Pxyi(X, idx))
            py_i = self.compute_Pxyi(X, idx) * self.phi[idx]
            Pyi[:, idx] = py_i
        return Pyi
    
    def predict(self, X):
        proba = self.predict_proba(X)
        class_indexes = np.argmax(proba, axis=1)
        
        # Replace index with class predictions
        vfunc = np.vectorize(lambda x: self.classes[x])
        class_predictions = vfunc(class_indexes)
        return class_predictions                           
    
    def generate_data(self, class_id, num_samples=1):
        """Generates new unseen dataset from a normal distribution
            given the mean of class and covariance
        """
        mean = self.mu[class_id]
        cov = self.sigma
        return np.random.multivariate_normal(mean, cov, num_samples)

In [6]:
GDA = GaussianDiscAnalysis()

In [7]:
GDA.fit(X_train, y_train)

In [8]:
predictions = GDA.predict(X_test)

In [9]:
predictions

array([1, 2, 1, 0, 0, 0, 0, 1, 2, 2, 1, 1, 0, 2, 0, 1, 1, 1, 0, 2, 0, 2,
       1, 2, 2, 2, 2, 0, 2, 0, 1, 1, 2, 0, 2, 0, 0, 1])

In [10]:
GDA.predict_proba(X_test)[0:5]

array([[0.    , 0.6263, 0.0008],
       [0.    , 0.    , 0.0266],
       [0.    , 0.0497, 0.    ],
       [0.7628, 0.    , 0.    ],
       [0.3123, 0.    , 0.    ]])

In [11]:
np.sum(predictions == y_test) / len(y_test)

0.9473684210526315

### Generate new data

In [12]:
new_data = GDA.generate_data(class_id=1, num_samples=20)

In [13]:
new_data

array([[5.8489, 2.5535, 4.4301, 1.55  ],
       [5.4008, 2.4028, 3.8874, 1.1854],
       [6.0701, 2.6925, 4.5285, 1.3446],
       [5.8933, 2.859 , 4.0502, 0.9833],
       [6.2032, 2.8221, 4.532 , 1.2805],
       [5.5509, 2.2722, 3.8771, 1.1105],
       [6.3919, 3.0769, 4.7419, 1.3295],
       [5.8338, 2.4944, 3.9456, 1.3381],
       [6.716 , 3.2511, 4.6925, 1.4022],
       [5.6894, 2.9112, 4.3122, 1.2708],
       [6.9438, 3.1577, 5.1802, 1.6474],
       [4.9971, 2.7765, 3.2524, 1.1737],
       [6.2053, 2.8841, 4.7763, 1.4664],
       [6.3208, 2.8905, 4.633 , 1.1944],
       [6.0029, 3.373 , 4.3417, 1.6442],
       [6.1169, 2.6744, 4.4568, 1.3319],
       [6.5776, 2.0728, 4.3906, 1.3364],
       [5.8233, 2.822 , 3.5306, 1.2569],
       [5.4096, 2.9752, 3.8528, 1.3642],
       [6.2788, 3.2446, 4.4207, 1.2901]])

In [14]:
GDA.predict(new_data)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])